In [1]:

from pathlib import Path
from zipfile import ZipFile
import pandas as pd
import geopandas as gpd
import fiona

raw_data = '2024_SAC_Secured_Public_Roll.zip'
raw_fname = 'secured_roll_public.txt'
parquet_file = None # '2024_SAC_Secured_Public_Roll.parquet' # specify path to parquet file. MUCH faster load time if already done

# parcel geom data
gpkg = 'parcel_gis.gpkg'
gpkg_lyr = 'Parcels'


#==========RUN SCRIPT AND SELDOM-CHANGED PARAMS============

# seldom-changed params
prop_tax_rate = 0.01 # assumed rate for entire property. Not changed. Will need to be made more sophisticated.

# load CSV from zip containing property value data for each parcel
f_propval = 'PROPVAL'
f_proptaxamt = 'PROPTAX'
f_landtaxamt = 'LANDTAX'
f_landval = 'LANDVAL'
f_impvts_val = 'IMP_VAL'

if parquet_file:
    df_master = gpd.read_parquet(parquet_file)
else:
    # if no parquet file available, then much slower load of data, especially if loading from GPKG or GeoJSON
    # load GIS data
    gdf_fields = ['APN', 'geometry', 'CITY', 'LU_GENERAL', 'LU_SPECIF', 'ZIP']
    gdf = gpd.read_file(gpkg, layer=gpkg_lyr, columns=gdf_fields)[gdf_fields]
    gdf.shape
    
    # make field names more intuitive
    renamer = {'LAND': f_landval, 'IM': 'IMP_VAL'}
    
    # specify cols to exclude to clean out clutter
    excl_cols = ['OWNER_CODE', 'OWNER', 'MAIL_ADDRESS',
           'MAIL_CITY', 'MAIL_STATE', 'MAIL_ZIP', 'CARE_OF', 'ZONING',
           'LAND_USE_CODE', 'RECORDING_DATE', 'RECORDING_PAGE', 'DEED_TYPE',
            'VALUE_DT', 'NGH', 'ACTION_CODE']
    
    with ZipFile(raw_data) as z:
       # open the csv file in the dataset
       with z.open(raw_fname) as f:
          # read the dataset
          df_taxdata = pd.read_csv(f, delimiter=';').rename(columns=renamer)
          df_taxdata['SITUS_NUMBER'] = df_taxdata['SITUS_NUMBER'].astype('str')
        
    # de-clutter
    df_taxdata = df_taxdata[[f for f in df_taxdata.columns if f not in excl_cols]]
    
    # create APN to enable join with GIS data
    df_taxdata['APN'] = df_taxdata['MAPB'].astype('str').str.pad(width=3, side='left', fillchar='0') \
            + df_taxdata['PG'].astype('str').str.pad(width=4, side='left', fillchar='0') \
            + df_taxdata['PCL'].astype('str').str.pad(width=3, side='left', fillchar='0') \
            + df_taxdata['PSUB'].astype('str').str.pad(width=4, side='left', fillchar='0')
    
    for f in ['PG', 'PCL', 'PSUB']: del df_taxdata[f]
    
    # merge prop val data to GIS data
    print(f"raw parcel gis data rows: {gdf.shape[0]}")
    df_master = gdf.merge(df_taxdata, on='APN')
    print(f"{gdf.shape[0] - df_master.shape[0]} rows removed during join.")
    
    # remove records where normal property tax is zero or less (assume to be special cases like hwy medians)
    df_master = df_master.loc[df_master[f_landval] > 0]
    print(f"{df_master.shape[0]} rows left after removing properties whose land value is zero.")
    
    # convert to CRS that can be mapped in plotly
    df_master = df_master.to_crs(4326)
    
    # free up memory
    del df_taxdata, gdf

# compute revenue-neutral land tax rate
df_master[f_propval] = df_master[f_landval] + df_master[f_impvts_val]
df_master[f_proptaxamt] = df_master[f_propval] * prop_tax_rate
prop_tax_revenue = df_master[f_proptaxamt].sum()
print('${:,.2f} total revenue from current property tax'.format(prop_tax_revenue))

# compute land tax rate
propv_landv_ratio = df_master[f_propval].sum() / df_master[f_landval].sum()
land_tax_rate = prop_tax_rate * propv_landv_ratio

# confirm that revenue from land tax = revenue from property tax using land tax rate
land_tax_rev = (df_master[f_landval] * land_tax_rate).sum()
diff = round(land_tax_rev - prop_tax_revenue, 2)
print(f"diff between prop tax revenue and LVT revenue (should be zero): {diff}")

# compute land tax amt for each parcel
f_taxdiff = 'TAXDIF'
df_master[f_landtaxamt] = df_master[f_landval] * land_tax_rate
df_master[f_taxdiff] = df_master[f_landtaxamt] - df_master[f_proptaxamt]

/var/folders/qn/87pf6mn56tv_yy_5djd7k32w0000gn/T/ipykernel_87029/3483247255.py:50: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_taxdata = pd.read_csv(f, delimiter=';').rename(columns=renamer)


raw parcel gis data rows: 479447
3730 rows removed during join.
460717 rows left after removing properties whose land value is zero.
$2,266,443,911.37 total revenue from current property tax
diff between prop tax revenue and LVT revenue (should be zero): 0.0


In [2]:
# Add census data to each parcel (e.g., allow aggregation by census tract?)

Index(['APN', 'geometry', 'CITY', 'LU_GENERAL', 'LU_SPECIF', 'ZIP', 'MAPB',
       'TAX_RATE_AREA', 'SITUS_NUMBER', 'SITUS_CITY', 'SITUS_STREET',
       'SITUS_ZIP', 'LANDVAL', 'IMP_VAL', 'FIXTURE', 'PP', 'HO_EX', 'EX',
       'ACTION_CODE                                                                                                                                             ',
       'PROPVAL', 'PROPTAX', 'LANDTAX', 'TAXDIF'],
      dtype='object')

In [4]:
# Export portion of full table
df_export = df_master[df_master['CITY'] == 'SACRAMENTO'].copy() # df_master.head(100)
export_name = 'SACRAMENTO'

# need to convert to string because otherwise will give mixed-type error when exporting to parquet
df_export['SITUS_NUMBER'] = df_export['SITUS_NUMBER'].astype('str') 

df_export.to_parquet(f'{export_name}.parquet')
print("successful output to parquet.")

successful output to parquet.


In [3]:
# Export full table to parquet (for faster loading in future)
df_master['SITUS_NUMBER'] = df_master['SITUS_NUMBER'].astype('str') 
df_master.to_parquet(f"{Path(raw_data).stem}.parquet")